In [1]:
import os 
os.chdir('..')

from utils import NestedDefaultdict
from loaders import DLTLoader

d = DLTLoader("logs/dlt/HPA_partnumber not updated.dlt")
msgs = d.get_messages()

In [2]:
import re

def sub_numeric_after_colon(text):
    pattern = r'((:|])\s)\b((0x)?[0-9A-Fa-f]+\b)'

    return re.sub(pattern, r'\1MASK', text)

def sub_perf_counter(text):
    time_units  = ['µs', 'microsecond', 'ms', 'millisecond', 's', 'second']

    pattern = r'(\d+)\s*(' + '|'.join(time_units) + r')'

    return re.sub(pattern, r'MASK\2', text, flags=re.IGNORECASE)


text = '''Some text: 0x1F3A:0x4B2F:0xA1C0: 0xABC123: 123214 
[aa:s: 123] Bus : control timer stop fail Routing info for remote service could not be found! (13ff): [01fe.0001.0001] 033f
warn: dap_swsl CheckLostComm: time_diff: 876928ms
warn: dap_swsl CheckLostComm: time_diff: 876928 ms 123 s
warn: dap_swsl CheckLostComm: time_diff: 876928 millisecond
warn: dap_swsl CheckLostComm: time_diff: 876928 milliseconds
'''


result = sub_perf_counter(sub_numeric_after_colon(text))
print(result)

Some text: MASK:0x4B2F:0xA1C0: MASK: MASK 
[aa:s: MASK] Bus : control timer stop fail Routing info for remote service could not be found! (13ff): [01fe.0001.0001] MASK
warn: dap_swsl CheckLostComm: time_diff: MASKms
warn: dap_swsl CheckLostComm: time_diff: MASK ms MASKs
warn: dap_swsl CheckLostComm: time_diff: MASK millisecond
warn: dap_swsl CheckLostComm: time_diff: MASK milliseconds



In [3]:
dlt = NestedDefaultdict(int, 5)


# with open("logs/dlt/compact_preprocess.txt", "w") as file:
for m in msgs:
    msg_type = m.type_string.decode('utf-8') 
    msg_subtype = m.subtype_string.decode('utf-8')
    if msg_type != 'log' \
        or msg_subtype not in ('warn', 'error', 'fatal'):
        continue 
    
    payload = sub_perf_counter(sub_numeric_after_colon(m.payload_decoded))
    dlt[m.ecuid][m.apid][m.ctid][m.seid][(msg_subtype,payload)] += 1 

In [4]:
counts = []
total = 0
with open("logs/dlt/compact_preprocess3.txt", "w") as file:
    for ecu_name, ecu in dlt.items():
        for app_name, app in ecu.items():
            for ctx_name, ctx in app.items():
                for ses_name, session in ctx.items():
                    counts.append(len(session))
                    block_name = f"{ecu_name} {app_name} {ctx_name} {ses_name}"
                    # print(block_name)
                    print(f"START {block_name}", file=file)
                    for (mtype, msg), count in session.items():
                        print(f"{count} {mtype}: {msg}", file=file)
                        total += count
                    print(f"END", file=file)

In [5]:
len(counts), sum(counts), total

(708, 2816, 376792)

In [6]:
len(counts), sum(counts)

(708, 2816)

In [7]:
2392 + 462*2

3316

In [8]:
import tiktoken

tokenizer = tiktoken.encoding_for_model("gpt-4")

In [9]:
text = 'END HPA KAW_ SFW 1061110'
len(tokenizer.encode(text))

12

In [10]:
from utils.text_splitters import DLTBlockSplitter
text_splitter = DLTBlockSplitter(8000)

with open("logs/dlt/compact_preprocess3.txt", "r") as file:
    texts = text_splitter.create_documents([file.read()])

In [11]:
texts, len(texts)

([Document(page_content='START HPA ZQF1 ZQF1 1061101\n34422 warn: bazel-out/k8-opt-ST-d9fd4f79f947/bin/deployment/z2_vcc_spa2/service_agents/communication/common/csp/_virtual_includes/csp_property_getter/common/csp_property_getter.h:35: Failed to get property value; group=feature_modes, error code=-1 (kFailure)\n1 warn: ./deployment/z2_vcc_spa2/service_agents/communication/common/csp/proxy_wrapper.h:127: Service adasfunctionsarbitrationsinterface is NOT available\n1 warn: ./deployment/z2_vcc_spa2/service_agents/communication/common/csp/proxy_wrapper.h:127: Service longitudinalvehiclecontrolobserver is NOT available\n1 warn: blocks/utilities/log_throttler/src/log_throttler.cpp:43: LongitudinalVehicleControlObserver not available\n9 warn: deployment/z2_vcc_spa2/service_agents/communication/vehicle_control_state/csp/vehicle_control_state_consumer_agent.cpp:91: Vehicle Control State deps not available. longitudinalvehiclecontrolobserver: MASK, adasmanagerstatusinterfacesafe: MASK, vehiclem

In [13]:
a = 1812+983+21
1812 / a, 983 / a, 21 / a

(0.6434659090909091, 0.34907670454545453, 0.007457386363636364)